# Programming Assignment: Логистическая регрессия

## Введение

Логистическая регрессия — один из видов линейных классификаторов. Одной из ее особенностей является возможность оценивания вероятностей классов, тогда как большинство линейных классификаторов могут выдавать только номера классов.

Логистическая регрессия использует достаточно сложный функционал качества, который не допускает записи решения в явном виде (в отличие от, например, линейной регрессии). Тем не менее, логистическую регрессию можно настраивать с помощью градиентного спуска.

Мы будем работать с выборкой, содержащей два признака. Будем считать, что ответы лежат в множестве {-1, 1}. Для настройки логистической регрессии мы будем решать следующую задачу:

![Image](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/eah308eQEeW7RxKvROGwrw_b19d35881f472bbcaaa2fb7fcf4813f7_logreg.png?expiry=1576281600000&hmac=MmO3imnKarO54-Hjq6CWPJHVZRXGR4VqmOl8g0PaxAU)

Здесь $x_{i1}$ и $x_{i2}$ — значение первого и второго признаков соответственно на объекте  $x_{i}$. В этом задании мы будем рассматривать алгоритмы без свободного члена, чтобы упростить работу.

Градиентный шаг для весов будет заключаться в одновременном обновлении весов $w_1$ и $w_2$ по следующим формулам (проверьте сами, что здесь действительно выписана производная нашего функционала):
![Image](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/hrIFu8eQEeWoXw5ZvVCRuw_1a61e227f5a1bc7fc0354df00fa70781_logderiv.png?expiry=1576281600000&hmac=iwzWoSW5zGPsHfFl1u_Z1hyZtILsGd5eJ9R1XcXwUqM)
![Image](https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/kb-c5MeQEeW7RxKvROGwrw_48f340a33cb38e1de9317b7df29d1cda_logderiv2.png?expiry=1576281600000&hmac=mDC4wi7-p8z8T8DF3NjaCONyml9RGSoK-cQ9a6i46RI)

Здесь $k$ — размер шага.

Линейные методы могут переобучаться и давать плохое качество из-за различных проблем в данных: мультиколлинеарности, зашумленности и т.д. Чтобы избежать этого, следует использовать регуляризацию — она позволяет понизить сложность модели и не допустить переобучения. Сила регуляризации определяется коэффициентом C в формулах, указанных выше.

## Реализация в Scikit-Learn

В этом задании мы предлагаем вам самостоятельно реализовать градиентный спуск.

В качестве метрики качества будем использовать `AUC-ROC` (`Area Under ROC-Curve`). Она предназначена для алгоритмов бинарной классификации, выдающих оценку принадлежности объекта к одному из классов. По сути, значение этой метрики является агрегацией показателей качества всех алгоритмов, которые можно получить, выбирая какой-либо порог для оценки принадлежности.

В `Scikit-Learn` метрика `AUC` реализована функцией `sklearn.metrics.roc_auc_score`. В качестве первого аргумента ей передается вектор истинных ответов, в качестве второго — вектор с оценками принадлежности объектов к первому классу.

## Материалы
* [Подробнее о логистической регрессии и предсказании вероятностей с ее помощью](https://github.com/esokolov/ml-course-hse/blob/master/2016-fall/lecture-notes/lecture05-linclass.pdf)
* [Подробнее о градиентах и градиентном спуске](https://github.com/esokolov/ml-course-hse/blob/master/2016-fall/lecture-notes/lecture02-linregr.pdf)

## Инструкция по выполнению

### Шаг 1: 

Загрузите данные из файла `data-logistic.csv`. Это двумерная выборка, целевая переменная на которой принимает значения -1 или 1.

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error as mse, roc_auc_score

In [2]:
df = pd.read_csv('data-logistic.csv', header=None)
X = df.loc[:, 1:]
y = df[0]

### Шаг 2: 

Убедитесь, что выше выписаны правильные формулы для градиентного спуска. Обратите внимание, что мы используем полноценный градиентный спуск, а не его стохастический вариант!

In [3]:
def sigma_y(i, w1, w2):
    return 1. / (1. + np.exp(-y[i] * (w1*X[1][i] + w2*X[2][i])))

def delta_for_w(w_index, w1, w2, C, k):
    addition = sum(y[i] * X[w_index][i] * (1. - sigma_y(i, w1, w2)) for i in np.arange(0, len(y)))
    addition *= k / len(y)
    addition -= k * C * (w1 if w_index == 1 else w2)
    
    return addition

### Шаг 3: 

Реализуйте градиентный спуск для обычной и `L2`-регуляризованной (с коэффициентом регуляризации 10) логистической регрессии. Используйте длину шага `k=0.1`. В качестве начального приближения используйте вектор (0, 0).

In [4]:
def gradient_regressor(C, iterations_remaining=10000, k=0.1, ERROR=1e-5):
    changed_w1, changed_w2 = 0., 0.
    while iterations_remaining:
        iterations_remaining -= 1
        w1, w2 = changed_w1, changed_w2
        changed_w1 = w1 + delta_for_w(1, w1, w2, C, k)
        changed_w2 = w2 + delta_for_w(2, w1, w2, C, k)
        if np.sqrt(mse([w1, w2], [changed_w1, changed_w2])) <= ERROR:
            break
    return changed_w1, changed_w2

### Шаг 4:

Запустите градиентный спуск и доведите до сходимости (евклидово расстояние между векторами весов на соседних итерациях должно быть не больше 1e-5). Рекомендуется ограничить сверху число итераций десятью тысячами.

In [5]:
def sigma(xi, w1, w2):
    return 1. / (1 + np.exp(-w1 * xi[1] - w2 * xi[2]))

w1, w2 = gradient_regressor(0.)
l2w1, l2w2 = gradient_regressor(10.)

print(w1, w2, l2w1, l2w2)

scores = X.apply(lambda xi: sigma(xi, w1, w2), axis=1)
l2scores = X.apply(lambda xi: sigma(xi, l2w1, l2w2), axis=1)

0.28768325128305977 0.09210201173422795 0.028558754546234223 0.024780137249735563


### Шаг 5: 

Какое значение принимает `AUC-ROC` на обучении без регуляризации и при ее использовании? Эти величины будут ответом на задание. В качестве ответа приведите два числа через пробел. Обратите внимание, что на вход функции roc_auc_score нужно подавать оценки вероятностей, подсчитанные обученным алгоритмом. Для этого воспользуйтесь сигмоидной функцией: $a(x) = 1 / (1 + exp(-w_1 x_1 - w_2 x_2)$).

In [6]:
auc_score = roc_auc_score(y, scores)
l2_auc_score = roc_auc_score(y, l2scores)

print(auc_score)
print(l2_auc_score)

with open("ans.txt", "w") as f:
    f.write(str(auc_score) + ' ' + str(l2_auc_score))

0.9268571428571428
0.9362857142857142


### Шаг 6: 

Попробуйте поменять длину шага. Будет ли сходиться алгоритм, если делать более длинные шаги? Как меняется число итераций при уменьшении длины шага?

### Шаг 7: 

Попробуйте менять начальное приближение. Влияет ли оно на что-нибудь?

Если ответом является нецелое число, то целую и дробную часть необходимо разграничивать точкой, например, 0.421. При необходимости округляйте дробную часть до трех знаков.